In [4]:
import os
import gc
import torch
import chromadb
from chromadb.utils import embedding_functions
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    Pipeline)
from sentence_transformers import CrossEncoder

from dotenv import load_dotenv
from openai import OpenAI
from typing import List, Dict
# Cleanup

gc.collect()
torch.cuda.empty_cache()
# Configuration
DB_PATH = os.path.join(os.getcwd(), "seamanuals")  # Safe path joining
MODEL_ID = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

In [5]:
load_dotenv()

class RAGGenerator:
    def __init__(self, api_key: str = None, base_url: str = None, model_name: str = "gpt-5-nano:free"):
        """
        Initialize the OpenAI-compatible client.

        Args:
            api_key: Your API Key (use "dummy" for local models like Ollama)
            base_url: The API endpoint (e.g., "http://localhost:11434/v1" for Ollama)
            model_name: The specific model to target (e.g., "llama3", "gpt-4o")
        """
        self.client = OpenAI(
            api_key=api_key or os.getenv("OPENAI_API_KEY"),
            base_url=base_url or os.getenv("OPENAI_BASE_URL")
        )
        self.model_name = model_name

    def construct_prompt(self, query: str, context_chunks: List[str]) -> str:
        """
        Builds the prompt by combining the user query with retrieved context.
        """
        # Join chunks with a clear separator
        context_str = "\n\n---\n\n".join(context_chunks)

        prompt = f"""You are a helpful assistant for maritime regulations.
Answer the user's question based ONLY on the following context.
If the answer is not in the context, say "I don't know."

The context may contain Markdown tables. Please interpret the rows and columns accurately.

### CONTEXT:
{context_str}

### USER QUESTION:
{query}

### ANSWER:
"""
        return prompt

    def generate_answer(self, query: str, context_chunks: List[str]) -> str:
        """
        Sends the prompt to the LLM and returns the response.
        """
        prompt = self.construct_prompt(query, context_chunks)

        try:
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=[
                    {"role": "system", "content": "You are a precise technical assistant."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1, # Keep strict for RAG to avoid hallucinations
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"Error during inference: {e}"


In [6]:
client = chromadb.PersistentClient(path=DB_PATH)

embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
collection = client.get_or_create_collection(name="Sea-Database",
                                             embedding_function=embedding_func)

In [17]:
def search(query: str, top_k_retrieval=20, top_k_rerank=5):

    # Stage 1: Semantic Retrieval (Bio-Encoder)
    results = collection.query(
        query_texts=[query],
        n_results=top_k_retrieval
    )
    documents = results['documents'][0]
    metadatas = results['metadatas'][0]
    # Stage 2: Re-ranking (Cross-Encoder)
    # Prepare pairs: (Query, Document_Context)
    pairs = [[query, doc] for doc in documents]

    # Predict scores
    scores = cross_encoder.predict(pairs)

    # Sort by score (descending)
    ranked_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)

    # Stage 3: Return top-k results
    retrieved = []
    for rank, idx in enumerate(ranked_indices[:top_k_rerank]):
        retrieved.append({
            "rank": rank+1,
            "score": scores[idx],
            "source": metadatas[idx]['source'],
            "page": metadatas[idx]['page'],
            "content": documents[idx]
        })

    return retrieved

In [8]:
print("⏳ Loading Cross-Encoder Model...")
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
print("✅ Cross-Encoder Loaded.")

⏳ Loading Cross-Encoder Model...
✅ Cross-Encoder Loaded.


In [19]:
query = "What are the specific carpet area requirements for classrooms in a Maritime Training Institute based on student intake capacity?"
docs = search(query)

print("--- Retrieved Context ---")
for doc in docs:
    print(f"Content \n: {doc['content']}")

--- Retrieved Context ---
Content 
: 2.9. Faculty room
2.9.1. A separate room not less than 8 m2 shall be provided for the Principal/head of
Institute. A carpet area of not less than 4 m2for each full-time faculty member shall be
provided. Modular separation between each faculty space is recommended.
2.9.2. The faculty shall also be provided with separate chair, table and cupboard.
Additional space, table and chairs shall be provided for visiting faculty. Ventilation and
lighting arrangement shall be same as for classrooms.
2.10. Classroom requirements
2.10.1. Class-room: The carpet area requirement of the class rooms and tutorial rooms
depends upon the number of students and type of seating arrangement. The size (carpet
area) of the classroom shall be 30 m2, 36 m2, and 50 m2 for intake capacity of 20, 24 and
40 candidates respectively. Institutes approved prior to 1st November, 2016 may continue
with the prevalent classroom size. However, if they apply for increase in capacity for
Con

In [28]:
rag = RAGGenerator(model_name='gemini-2.5-flash')

In [29]:
# 1. SIMULATE RETRIEVAL
query = "What are the specific carpet area requirements for classrooms in a Maritime Training Institute based on student intake capacity?"
docs = search(query)
context = ["-> ".join([doc['source'], doc['content']]) for doc in docs]
# 2. GENERATE RESPONSE
print(f"Query: {query}\n")
answer = rag.generate_answer(query, context)
print(f"Response:\n{answer}")

Query: What are the specific carpet area requirements for classrooms in a Maritime Training Institute based on student intake capacity?

Response:
The carpet area requirements for classrooms based on student intake capacity are:

*   **20 candidates:** 30 m²
*   **24 candidates:** 36 m²
*   **40 candidates:** 50 m²

Institutes approved before November 1, 2016, may continue with their existing classroom sizes, but if they apply for an increase in capacity or approval of new courses, they must comply with these latest guidelines.
